In [1]:
import pandas as pd
import numpy as np
# Working with dates
from datetime import datetime, timedelta

# Data download and APIs
import requests
import io
import zipfile
import os

# Warnings & progress
import warnings
import time

In [2]:
raw_weather = pd.read_parquet('data/daily_weather.parquet',engine='pyarrow')
print(f"{len(raw_weather):,} rows in raw weather data")
raw_weather.head()

27,635,763 rows in raw weather data


,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,41515,Asadabad,1957-07-01,Summer,27.0,21.1,35.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,41515,Asadabad,1957-07-02,Summer,22.8,18.9,32.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,41515,Asadabad,1957-07-03,Summer,24.3,16.7,35.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,41515,Asadabad,1957-07-04,Summer,26.6,16.1,37.8,4.1,NaN,NaN,NaN,NaN,NaN,NaN
4,41515,Asadabad,1957-07-05,Summer,30.8,20.0,41.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
phx_raw = raw_weather[raw_weather['city_name'].str.lower() == 'phoenix']
phx_raw.head()

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,72278,Phoenix,1933-06-01,Summer,NaN,13.3,36.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72278,Phoenix,1933-06-02,Summer,NaN,18.9,37.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72278,Phoenix,1933-06-03,Summer,NaN,21.7,35.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72278,Phoenix,1933-06-04,Summer,NaN,12.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72278,Phoenix,1933-06-05,Summer,NaN,17.2,33.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
phx_raw['date'] = pd.to_datetime(phx_raw['date'])
phx_raw_recent = phx_raw[phx_raw['date'].dt.year >= 2010]
phx_raw_recent = phx_raw_recent.drop_duplicates()
phx_raw_recent = phx_raw_recent.sort_values('date')
print(phx_raw.info())
print(phx_raw_recent.isnull().sum())
phx_raw_recent.head()

<class 'pandas.core.frame.DataFrame'>
Index: 31073 entries, 0 to 31072
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   station_id              31073 non-null  category      
 1   city_name               31073 non-null  category      
 2   date                    31073 non-null  datetime64[us]
 3   season                  31073 non-null  category      
 4   avg_temp_c              18495 non-null  float64       
 5   min_temp_c              31069 non-null  float64       
 6   max_temp_c              31070 non-null  float64       
 7   precipitation_mm        31017 non-null  float64       
 8   snow_depth_mm           20937 non-null  float64       
 9   avg_wind_dir_deg        5502 non-null   float64       
 10  avg_wind_speed_kmh      18505 non-null  float64       
 11  peak_wind_gust_kmh      11739 non-null  float64       
 12  avg_sea_level_pres_hpa  18222 non-null  float64    

/var/folders/cj/m80dzr_130sg7k1csqxc0tw00000gn/T/ipykernel_1703/3722206636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phx_raw['date'] = pd.to_datetime(phx_raw['date'])


,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
26081,72278,Phoenix,2010-01-01,Winter,12.5,5.0,20.0,0.0,NaN,NaN,5.0,NaN,1022.2,NaN
26082,72278,Phoenix,2010-01-02,Winter,14.3,7.2,21.7,0.0,NaN,NaN,4.3,NaN,1017.4,NaN
26083,72278,Phoenix,2010-01-03,Winter,13.6,5.6,21.7,0.0,NaN,NaN,3.6,NaN,1018.4,NaN
26084,72278,Phoenix,2010-01-04,Winter,15.8,7.2,23.3,0.0,NaN,NaN,9.4,NaN,1018.4,NaN
26085,72278,Phoenix,2010-01-05,Winter,14.7,7.2,22.8,0.0,NaN,NaN,2.2,NaN,1017.0,NaN


In [9]:
phx_raw_recent.to_csv('data/phx_weather_2010_2023.csv', index=False)